# Менеджеры контекста

## Что это такое

In [ ]:
import datetime
with open('tmp.txt', 'w') as f:
    f.write('Привет всем. Текущее время ' + str(datetime.datetime.now()))
    
# Эквивалентные конструкции
f = open('tmp.txt', 'r')
print(f.read())
f.close()
print(f.closed)

In [ ]:
# Но не совсем
f = open('tmp.txt', 'r')
print(f.read())
raise(ValueError)
f.close()

In [ ]:
print(f.closed)

In [ ]:
f.close()

## Как реализовать свой

In [ ]:
class File(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        # Здесь может быть сложная логика обработки исключения
        self.file_obj.close()

In [ ]:
with File('tmp.txt', 'r') as f:
    print(f.read())
    raise(ValueError)

In [ ]:
print(f.closed)

### Это не только файлы и соединения

Транзакция

```python
from django.db import transaction

def viewfunc(request):
    # This code executes in autocommit mode (Django's default).
    do_stuff()

    with transaction.atomic():
        # This code executes inside a transaction.
        do_more_stuff()
```

Блокировка

```python
from filelock import Timeout, FileLock

lock = FileLock("high_ground.txt.lock")
with lock:
    open("high_ground.txt", "a").write("You were the chosen one.")
```

## contextlib

In [1]:
from contextlib import contextmanager

@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()

In [2]:
with closing(open('tmp.txt', 'r')) as f:
    print(f.read())
print(f.closed)

IOError: [Errno 2] No such file or directory: 'tmp.txt'

In [ ]:
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('http://ip.jsontest.com/')) as page:
    for line in page:
        print(line)

In [ ]:
import os
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove('somefile.tmp')

# Работа с файлами

## Базовые операции

In [ ]:
import datetime
with open('tmp.txt', 'w') as f:
    f.write('Привет всем. Текущее время ' + str(datetime.datetime.now()))
    
with open('tmp.txt', 'r') as f:
    print(f.read())

In [ ]:
# Дописываем в конец
with open('tmp.txt', 'a') as f:
    for _ in range(5):
        f.write('\nПривет всем. Текущее время ' + str(datetime.datetime.now()))
        
with open('tmp.txt', 'r') as f:
    for line in f: # читаем построчно
        print(line.strip())

In [ ]:
# читаем целиком
with open('tmp.txt', 'r') as f:
    print(f.read())

In [ ]:
with open('tmp.txt', 'r') as f:
    f.seek(25) # Переходим на нужню позицию
    print(f.read(30)) # Читаем нужное количество байт

## Кодировки

* https://ru.wikipedia.org/wiki/ASCII
* https://ru.wikipedia.org/wiki/Windows-1251
* https://ru.wikipedia.org/wiki/UTF-8
* https://ru.wikipedia.org/wiki/UTF-16

https://ru.wikipedia.org/wiki/Юникод - это не кодировка!


In [3]:
# Пишем в конкретной кодировке
with open('tmp.txt', 'w', encoding='cp1251') as f:
    f.write('Привет всем. Текущее время ' + str(datetime.datetime.now()))

TypeError: 'encoding' is an invalid keyword argument for this function

In [ ]:
with open('tmp.txt', 'r') as f:
    print(f.read())

In [ ]:
# Ошиблись в кодировке при чтении
with open('tmp.txt', 'r', encoding='cp1252') as f:
    print(f.read())

In [ ]:
# Можем прочитать как byte
with open('tmp.txt', 'rb') as f:
    cont = f.read()
print(cont)
type(cont)

In [ ]:
# И затем декодировать
cont_str = cont.decode('cp1251')
print(cont_str)
type(cont_str)

In [ ]:
# И закодировать в другую кодировку
bytes(cont_str, 'utf8')

In [ ]:
# Кодировка по-умолчанию - utf8
bytes(cont_str, 'utf8').decode()

## Материалы по теме

Статья про кодировки на хабре: [Что нужно знать каждому разработчику о кодировках и наборах символов для работы с текстом](https://habrahabr.ru/post/158639/)

И сразу про нормализацию строк: [«Й» вам не «и» краткое! О важности нормализации Unicode](https://habrahabr.ru/post/262679/)

## Bytes

In [4]:
# могут интерпретироваться как строки
b = bytes(cont_str, 'utf8')
print(b[1:5])
print(b.split())

NameError: name 'cont_str' is not defined

In [ ]:
# и как последовательности чисел
print(b[5])
print(list(b))

In [ ]:
# иммутабельны как и строки
b[4] = '6'

In [ ]:
# но есть и мутабельная версия
ba = bytearray(b)
print(ba[:5])

In [ ]:
# хоть в этом и мало смысла
ba[5] += 1
print(ba.decode())

In [ ]:
# для чего-то низкоуровнего
ba = bytearray(range(50))
print(ba)
print(list(ba))

# Бинарные форматы

In [5]:
import struct

bmp = open("VENUS.bmp", 'rb')
print('Type:', bmp.read(2).decode())
print('Size: %s' % struct.unpack('I', bmp.read(4)))
print('Reserved 1: %s' % struct.unpack('H', bmp.read(2)))
print('Reserved 2: %s' % struct.unpack('H', bmp.read(2)))
print('Offset: %s' % struct.unpack('I', bmp.read(4)))

print('DIB Header Size: %s' % struct.unpack('I', bmp.read(4)))
print('Width: %s' % struct.unpack('I', bmp.read(4)))
print('Height: %s' % struct.unpack('I', bmp.read(4)))
print('Colour Planes: %s' % struct.unpack('H', bmp.read(2)))
print('Bits per Pixel: %s' % struct.unpack('H', bmp.read(2)))
print('Compression Method: %s' % struct.unpack('I', bmp.read(4)))
print('Raw Image Size: %s' % struct.unpack('I', bmp.read(4)))
print('Horizontal Resolution: %s' % struct.unpack('I', bmp.read(4)))
print('Vertical Resolution: %s' % struct.unpack('I', bmp.read(4)))
print('Number of Colours: %s' % struct.unpack('I', bmp.read(4)))
print('Important Colours: %s' % struct.unpack('I', bmp.read(4)))

('Type:', u'BM')
Size: 46078
Reserved 1: 0
Reserved 2: 0
Offset: 1078
DIB Header Size: 40
Width: 300
Height: 150
Colour Planes: 1
Bits per Pixel: 8
Compression Method: 0
Raw Image Size: 45000
Horizontal Resolution: 0
Vertical Resolution: 0
Number of Colours: 256
Important Colours: 256


In [9]:
import struct

bmp = open("VENUS.bmp", 'rb')
print('Type:', bmp.read(2).decode())
tpl = struct.unpack('IHHII', bmp.read(16))
print(tpl)
print('Size: {}; Reserved 1: {}; Reserved 2: {}; Offset {}'.format(*tpl))


('Type:', u'BM')
(46078, 0, 0, 1078, 40)
Size: 46078; Reserved 1: 0; Reserved 2: 0; Offset 1078


In [12]:
import struct
example = 'Hello world'
ln = len(example)
enc = struct.pack('I{}s'.format(ln), ln, example.encode())
print(enc)

   Hello world


In [11]:
ln = struct.unpack_from('I', enc)[0]
print(ln)
enc = enc[struct.calcsize('I'):]
example = struct.unpack_from('{}s'.format(ln), enc)[0]
print(example)

11
Hello world


## Pickle

In [13]:
import pickle

class A:
    fld = 123
    
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __repr__(self):
        return 'A({0.a},{0.b})'.format(self)
        
t1 = [1, {'a': (1, 2, 3), 'b': set([1,2,3,4,5,4,3,2,1])}, 3, A(4, 5)]
s = pickle.dumps(t1)
print(s)
t2 = pickle.loads(s)
print(t2)

(lp0
I1
a(dp1
S'a'
p2
(I1
I2
I3
tp3
sS'b'
p4
c__builtin__
set
p5
((lp6
I1
aI2
aI3
aI4
aI5
atp7
Rp8
saI3
a(i__main__
A
p9
(dp10
g2
I4
sg4
I5
sba.
[1, {'a': (1, 2, 3), 'b': set([1, 2, 3, 4, 5])}, 3, A(4,5)]


# Файловая система

In [2]:
import os # почитайте что там есть - очень полезно для скриптов
os.path.exists('/tmp')

True

In [3]:
from glob import glob
glob('../*/*.ipynb')

['../02_data_structures/notebook.ipynb',
 '../03_oop/main.ipynb',
 '../07_async/main.ipynb',
 '../01_intro/main.ipynb',
 '../04_io/main.ipynb',
 '../05_meta/main.ipynb']

In [4]:
for t in os.walk('../'):
    print(t)

('../', ['06_db', '02_data_structures', '03_oop', '08_django', '07_async', '01_intro', '04_io', '05_meta'], [])
('../06_db', [], ['\xd0\x9b\xd0\xb5\xd0\xba\xd1\x86\xd0\xb8\xd1\x8f 6.pptx'])
('../02_data_structures', [], ['notebook.ipynb'])
('../03_oop', ['import_sample', 'test_sample', '.ipynb_checkpoints'], ['main.ipynb'])
('../03_oop/import_sample', ['geometry'], ['predicates.py'])
('../03_oop/import_sample/geometry', [], ['square.py', 'circle.py', '__init__.py'])
('../03_oop/test_sample', ['tests'], ['interval.py'])
('../03_oop/test_sample/tests', [], ['test_interval.py', '__init__.py'])
('../03_oop/.ipynb_checkpoints', [], ['main-checkpoint.ipynb'])
('../08_django', [], ['08-django.pdf'])
('../07_async', [], ['server.py', 'main.ipynb'])
('../01_intro', ['.ipynb_checkpoints'], ['main.ipynb'])
('../01_intro/.ipynb_checkpoints', [], ['main-checkpoint.ipynb'])
('../04_io', ['.ipynb_checkpoints'], ['http_server.py', '\xd0\x9a\xd0\xbe\xd0\xbd\xd1\x81\xd0\xbf\xd0\xb5\xd0\xba\xd1\x82_4', '

In [5]:
for curr_dir, sub_dirs, files  in os.walk('../'):
    for file in files:
        if file.endswith('.ipynb'):
            print(os.path.join(curr_dir, file))

../02_data_structures/notebook.ipynb
../03_oop/main.ipynb
../03_oop/.ipynb_checkpoints/main-checkpoint.ipynb
../07_async/main.ipynb
../01_intro/main.ipynb
../01_intro/.ipynb_checkpoints/main-checkpoint.ipynb
../04_io/main.ipynb
../04_io/.ipynb_checkpoints/main-checkpoint.ipynb
../05_meta/main.ipynb


## Requests

In [ ]:
# установим пакет
# !pip install requests

In [8]:
import requests
import json
resp = requests.get('http://date.jsontest.com/')
print(resp, resp.ok, resp.status_code)
print(resp.content)
print(json.loads(resp.content))

(<Response [200]>, True, 200)
{
   "time": "04:46:19 PM",
   "milliseconds_since_epoch": 1540572379007,
   "date": "10-26-2018"
}

{u'date': u'10-26-2018', u'milliseconds_since_epoch': 1540572379007, u'time': u'04:46:19 PM'}


In [9]:
print(resp.json())

{u'date': u'10-26-2018', u'milliseconds_since_epoch': 1540572379007, u'time': u'04:46:19 PM'}


# Многопоточность

## Потоки. Потоки здорового человека

In [15]:
%%time
# просто пример отправки данных - так в жизни делать не надо)
base_url = "http://md5.jsontest.com/?text="
import requests

def slow_md5(data):
    response = requests.get(base_url + data)
    if response.ok:
        rez = response.json()
        return rez.get('md5')

print(slow_md5('test1'))

5a105e8b9d40e1329780d62ea2265d8a
CPU times: user 6 ms, sys: 588 µs, total: 6.59 ms
Wall time: 355 ms


In [ ]:
test_strings = []
for i in range(50):
    test_strings.append('test_'+str(i))

In [23]:
%%time
rez = {}
for data in test_strings:
    rez[data] = slow_md5(data)

CPU times: user 343 ms, sys: 39.3 ms, total: 383 ms
Wall time: 16.3 s


In [25]:
%%time
import threading
threads_num = 4

rez = {}
def thread_run(idx):
    while idx < len(test_strings):
        data = test_strings[idx]
        rez[data] = slow_md5(data)
        idx += threads_num
    print('Thread end:'+str(idx))

threads = []
for i in range(threads_num):
    t = threading.Thread(target=thread_run, args=(i,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()
    

Thread end:51
Thread end:50
Thread end:52
Thread end:53
CPU times: user 373 ms, sys: 67.3 ms, total: 440 ms
Wall time: 5.73 s


In [26]:
import sys
sys.version

'2.7.15rc1 (default, Apr 15 2018, 21:51:34) \n[GCC 7.3.0]'

## Потоки. Потоки курильщика

In [1]:
def slow_fib(n):
    if n == 0:
        return 0
    if n < 3:
        return 1
    return slow_fib(n - 1) + slow_fib(n - 2)

print(slow_fib(10))

55


In [2]:
test_ns = []
for i in range(35):
    test_ns.append(i)

In [29]:
%%time
rez = {}
for n in test_ns:
    rez[n] = slow_fib(n)

CPU times: user 3.5 s, sys: 709 µs, total: 3.5 s
Wall time: 3.49 s


In [32]:
%%time
import threading
threads_num = 4

rez = {}
# простое определение потока
def thread_run(idx):
    while idx < len(test_ns):
        data = test_ns[idx]
        rez[data] = slow_fib(data)
        idx += threads_num
    print('Thread end: ' + str(idx))

threads = []
for i in range(threads_num):
    t = threading.Thread(target=thread_run, args=(i,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()
    

Thread end: 35
Thread end: 36
Thread end: 37
Thread end: 38
CPU times: user 7.74 s, sys: 2.84 s, total: 10.6 s
Wall time: 6.13 s


## GIL
- https://wiki.python.org/moin/GlobalInterpreterLock
- https://habrahabr.ru/post/84629/

## Что делать
- numpy http://www.numpy.org
- процессы
- Py_BEGIN_ALLOW_THREADS в C extension [документация](http://www.cmi.ac.in/~madhavan/courses/prog2-2015/docs/python-3.4.2-docs-html/c-api/init.html)

## Процессы

In [35]:
%%time
import multiprocessing

# Выделяем пул процессов
p = multiprocessing.Pool(processes=6)

# Запускаем параллельную работу
result = p.map(slow_fib, test_ns)
print(result)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]
CPU times: user 0 ns, sys: 26.2 ms, total: 26.2 ms
Wall time: 1.79 s


In [36]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()

hello


In [37]:
# требует аккуратного проектирования (если бы не было timeout - зависло бы навсегда)
q.get(timeout=2)

Empty: 

## Модуль subprocess

In [1]:
import subprocess
pr = subprocess.run('date')

AttributeError: 'module' object has no attribute 'run'

In [2]:
pr.stdout

NameError: name 'pr' is not defined

In [ ]:
pr = subprocess.run('date', stdout=subprocess.PIPE)
pr

In [ ]:
print(pr.stdout.decode())

In [ ]:
subprocess.run('ls -l', stdout=subprocess.PIPE)
# subprocess.run(['ls', '-l'], stdout=subprocess.PIPE)

In [ ]:
pr = subprocess.Popen('sort', stdout=subprocess.PIPE, stdin=subprocess.PIPE)
for i in range(25):
    pr.stdin.write(f'{i}\n'.encode())
out, err = pr.communicate()
print(out.decode())
pr.terminate()

In [ ]:
pr.returncode

https://amoffat.github.io/sh/ - более удобный способ

# Работа с сетью

https://ru.wikipedia.org/wiki/TCP/IP

## Сокеты

In [ ]:
import socket
def listen():
    connection = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    connection.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    connection.bind(('0.0.0.0', 5555))
    connection.listen(10)
    while True:
        current_connection, address = connection.accept()
        while True:
            data = current_connection.recv(2048)

            if data == b'quit\n':
                current_connection.shutdown(1)
                current_connection.close()
                break
            elif data == b'stop\n':
                current_connection.shutdown(1)
                current_connection.close()
                return
            elif data:
                current_connection.send(data.upper())
                
listen()

## HTTP под капотом

In [5]:
import socket
import sys
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = 'md5.jsontest.com'
port = 80
s.connect((host, port))
s.send(b'GET /?text=test1 HTTP/1.1\n')
s.send(b'host: md5.jsontest.com\n')
s.send(b'\n\n')
data = s.recv(1000000) 
print(data.decode())
s.close()

HTTP/1.1 200 OK
Access-Control-Allow-Origin: *
Content-Type: application/json; charset=ISO-8859-1
X-Cloud-Trace-Context: f0b2fef7e96d382d21f7956250fb0c2a
Date: Fri, 26 Oct 2018 17:26:59 GMT
Server: Google Frontend
Content-Length: 73

{
   "md5": "5a105e8b9d40e1329780d62ea2265d8a",
   "original": "test1"
}



In [6]:
import time
from http.server import BaseHTTPRequestHandler, HTTPServer

HOST_NAME = 'localhost'
PORT_NUMBER = 8080

class MyHandler(BaseHTTPRequestHandler):
    def do_HEAD(s):
        s.send_response(200)
        s.send_header("Content-type", "text/html")
        s.end_headers()
    def do_GET(s):
        """Respond to a GET request."""
        s.send_response(200)
        s.send_header("Content-type", "text/html")
        s.end_headers()
        s.wfile.write(b"<html><head><title>Title goes here.</title></head>")
        s.wfile.write(b"<body><p>This is a test.</p>")

        s.wfile.write(b"<p>You accessed path: {}</p>".format(s.path.encode()))
        s.wfile.write(b"</body></html>")

if __name__ == '__main__':
    server_class = HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime(), f"Server Starts - {HOST_NAME}:{PORT_NUMBER}")
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    print(time.asctime(), f"Server Stops - {HOST_NAME}:{PORT_NUMBER}")

SyntaxError: invalid syntax (<ipython-input-6-6d148ce36fcc>, line 26)

## HTTP с фреймворком

Простой веб-сервер на Tornado
http://www.tornadoweb.org/en/stable/web.html - Документация

```python
import tornado.ioloop
import tornado.web

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.write("Hello, world")

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler),
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8080)
    tornado.ioloop.IOLoop.current().start()
```



## Домашнее задание
Сервер очередей заданий на базе TCP сокетов - task_queue